<a href="https://colab.research.google.com/github/Gemlala/AI-Projekt/blob/woche2/Kreisel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 1. Sicherstellen, dass wir im richtigen Ordner sind
%cd /content/drive/MyDrive/AI-Projekt

# 2. Status prüfen (du solltest sehen, dass Kreisel.ipynb geändert wurde)
!git status

/content/drive/MyDrive/AI-Projekt
On branch woche2
nothing to commit, working tree clean


In [ ]:
import os
import glob
from PIL import Image
from tqdm.auto import tqdm # Import für den Fortschrittsbalken

# Pfade
input_tiles = "/content/drive/MyDrive/Colab Notebooks/Bsc/SwissImageTiles_Kreisel_0.1m"
yolo_root = "/content/drive/MyDrive/Colab Notebooks/Bsc/YOLO_Dataset"

os.makedirs(f"{yolo_root}/images", exist_ok=True)
os.makedirs(f"{yolo_root}/labels", exist_ok=True)

tif_files = glob.glob(os.path.join(input_tiles, "*.tif"))

print(f"Starte Label-Generierung für {len(tif_files)} Kreisel-Bilder...")

# Wir packen die Schleife in tqdm()
for tif_path in tqdm(tif_files, desc="Konvertierung & Labeling"):
    file_name = os.path.basename(tif_path)
    base_name = os.path.splitext(file_name)[0]

    # Pfad für das Ziel-Bild prüfen, um Zeit zu sparen (Skip falls existiert)
    target_img_path = os.path.join(yolo_root, "images", f"{base_name}.jpg")

    if not os.path.exists(target_img_path):
        # 1. Bild konvertieren (TIF -> JPG)
        with Image.open(tif_path) as img:
            rgb_img = img.convert('RGB')
            rgb_img.save(target_img_path, quality=95)

    # 2. YOLO Label erstellen
    with open(os.path.join(yolo_root, "labels", f"{base_name}.txt"), "w") as f:
        f.write("0 0.5 0.5 0.8 0.8\n")

print(f"\n✅ Fertig! Datensatz bereit in: {yolo_root}")

In [ ]:
%pip install ultralytics
from ultralytics import YOLO

In [ ]:
import yaml

data_config = {
    'train': '/content/drive/MyDrive/Colab Notebooks/Bsc/YOLO_Dataset/images',
    'val': '/content/drive/MyDrive/Colab Notebooks/Bsc/YOLO_Dataset/images', # Für den Anfang nutzen wir dieselben Bilder
    'nc': 1,
    'names': ['Kreisel']
}

with open('/content/drive/MyDrive/Colab Notebooks/Bsc/YOLO_Dataset/data.yaml', 'w') as f:
    yaml.dump(data_config, f)

In [ ]:
# 1. Lade ein vortrainiertes Nano-Modell (schnell und effizient)
model = YOLO("yolo11n.pt")

# 2. Starte das Training
results = model.train(
    data='/content/drive/MyDrive/Colab Notebooks/Bsc/YOLO_Dataset/data.yaml',
    epochs=50,        # Wie oft das Modell den Datensatz durcharbeitet
    imgsz=1024,       # Deine Kachelgröße
    batch=8,          # Abhängig vom GPU-Speicher (in Colab meist 8-16)
    name='kreisel_modell_v1',
    device=0          # Nutzt die GPU
)

In [ ]:
from IPython.display import Image, display
import os

# Pfad zum neuesten Trainings-Lauf (YOLO nummeriert diese: v1, v2, etc.)
latest_run = "/content/runs/detect/kreisel_modell_v1/"

# 1. Lernkurven anzeigen
print("Trainings-Statistiken (Loss & Genauigkeit):")
display(Image(filename=f"{latest_run}results.png", width=800))

In [ ]:
print("Confusion Matrix (Was verwechselt das Modell?):")
display(Image(filename=f"{latest_run}confusion_matrix.png", width=600))

In [ ]:
# Zeige die ersten Vorhersagen auf den Testbildern
print("Beispiel-Vorhersagen des Modells:")
display(Image(filename=f"{latest_run}val_batch0_labels.jpg", width=800)) # Die echten Labels
display(Image(filename=f"{latest_run}val_batch0_pred.jpg", width=800))   # Die Vorhersagen der KI

In [ ]:
import os
import pandas as pd
from tqdm import tqdm
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction

# 1. Pfade konfigurieren
# Ersetze diesen Pfad durch den Ort deiner 'best.pt' aus dem Training
model_path = "/content/runs/detect/kreisel_modell_v1/weights/best.pt"
input_dir = "/content/drive/MyDrive/Colab Notebooks/Bsc/SwissImageTiles_40k/"
output_csv = "/content/drive/MyDrive/Colab Notebooks/Bsc/inventar_kreisel_schweiz.csv"

# 2. Dein spezialisiertes Modell laden
detection_model = AutoDetectionModel.from_pretrained(
    model_type="yolov8", # SAHI nutzt für YOLOv11 denselben Typ
    model_path=model_path,
    confidence_threshold=0.35, # Nur sichere Funde
    device="cuda",             # Nutzt die GPU für Speed
)

results_list = []
all_images = [f for f in os.listdir(input_dir) if f.endswith(".tif")]

print(f"Starte Inferenz für {len(all_images)} Bilder...")

# 3. Massenverarbeitung mit Fortschrittsbalken
for img_name in tqdm(all_images):
    img_path = os.path.join(input_dir, img_name)

    try:
        # Sliced Prediction (optimiert für 10.000x10.000 Pixel)
        result = get_sliced_prediction(
            img_path,
            detection_model,
            slice_height=1024,
            slice_width=1024,
            overlap_height_ratio=0.2,
            overlap_width_ratio=0.2,
            verbose=0 # Schaltet Konsolen-Spam aus
        )

        # Funde in die Liste schreiben
        for pred in result.object_prediction_list:
            bbox = pred.bbox.to_xyxy()
            results_list.append({
                "tile_name": img_name,
                "confidence": pred.score.value,
                "x_min": bbox[0],
                "y_min": bbox[1],
                "x_max": bbox[2],
                "y_max": bbox[3],
                "center_x": (bbox[0] + bbox[2]) / 2,
                "center_y": (bbox[1] + bbox[3]) / 2
            })

    except Exception as e:
        print(f"Fehler bei Bild {img_name}: {e}")

# 4. Ergebnisse sichern
df = pd.DataFrame(results_list)
df.to_csv(output_csv, index=False)

print(f"\nVerarbeitung abgeschlossen!")
print(f"Gefundene Kreisel-Kandidaten: {len(df)}")
print(f"Ergebnisse gespeichert in: {output_csv}")